# Applying prediction model to data accumulated over the whole city

This notebook runs the same architecture as sensor_rnn_model and sensor_rnn_sliding_window except applied to a different dataset. In thes dataset used here, all the footfalls are accumulated at each timestep. This is in order to try and predict the footfall for the entire city at one time.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.metrics import accuracy_score, precision_score, recall_score, mean_absolute_error, mean_absolute_percentage_error, r2_score, mean_squared_error
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Model
from tensorflow.keras.layers import *
from tensorflow.keras import backend as K
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping

import sklearn
from sklearn.inspection import permutation_importance

from sklearn.utils import class_weight

import os
import numpy as np

from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense, Dropout, Embedding, Masking, Bidirectional
from keras.optimizers import Adam

from keras.utils import plot_model

from datetime import datetime
from datetime import timedelta

#!pip install seaborn
#import seaborn as sns

## Load and prepare the data

In [2]:
def prepare_x_y_data(input_csv):
    # Read in formatted data
    data = pd.read_csv(input_csv, index_col = False)
    data = data.fillna(0)
    
    ### Delete unneeded columns - we currently include data from all sensors (even incomplete ones)
    sensor_ids = data['sensor_id']
    #data = data.drop(['sensor_id'],axis=1) # don't want this included
    # Get rid of columns in which none of the sensors have a value
    for column in data.columns:
        if np.nanmax(data[column]) ==0:
            del data[column]
            
    # Filter columns using the regex pattern in function input
    regex_pattern = 'buildings$|street_inf$|landmarks$'
    data = data[data.columns.drop(list(data.filter(regex=regex_pattern)))].copy()
    
    ### Add a random variable (to compare performance of other variables against)
    rng = np.random.RandomState(seed=42)
    data['random'] = np.random.random(size=len(data))
    data["random_cat"] = rng.randint(3, size=data.shape[0])
    
    ## Prepare data for modelling 
    ### Split into predictor/predictand variables
    Xfull = data.drop(['hourly_counts'], axis =1)
    Yfull = data['hourly_counts'].values
       
    ### Store the (non Sin/Cos) time columns and then remove them (Need them later to segment the results by hour of the day)
    data_time_columns = Xfull[['day_of_month_num', 'time', 'weekday_num', 'time_of_day', 'datetime']]
    #Xfull = Xfull.drop(['day_of_month_num', 'time', 'weekday_num', 'time_of_day','datetime', 'month_num'],axis=1)
    Xfull = Xfull.drop(['day_of_month_num', 'time', 'weekday_num', 'time_of_day', 'month_num'],axis=1)
    return Xfull, Yfull, data_time_columns

In [3]:
# data normalizing function
def normalize(df, target_column):
    
    result = df.copy()
    
    for feature_name in df.columns:
        
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()
        
        if feature_name == 'footfall':
            result['footfall_norm'] = (df[feature_name] - min_value) / (max_value - min_value)
            
        else:
            result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
    
    cols = list(result.columns)
    column_list = cols[:-2] + cols[-1:] + cols[-2:-1]
    result = result[column_list]
    
    return result

In [4]:
# load in the raw data - see .README file for info on the raw data
buffer_size_m = 400
input_csv ="/lustre_scratch/eejasm/DUST/formatted_data_for_modelling_allsensors_{}.csv".format(buffer_size_m)

X_data, Y_data, data_time_columns = prepare_x_y_data(input_csv)

X_data = X_data.iloc[:2198889]
Y_data = Y_data[:2198889]
data_time_columns = data_time_columns.iloc[:2198889]

In [5]:
# create df to combine the x and y data
df = X_data.copy()

df = df.loc[:,['datetime', 'Temp', 'Humidity' ,'Pressure', 'Rain', 'WindSpeed', 'Rainfall amount (millimetres)',
               'Sin_time','Cos_time','Sin_month_num','Cos_month_num','Sin_weekday_num','Cos_weekday_num']] 

df['footfall'] = Y_data

In [6]:
# need to accumulate the footfall number at each sensor for each timestep
# first find all the unique timesteps
city_df = df.iloc[:,:-1].drop_duplicates(keep='first').reset_index(drop=True)
# then  group the timesteps and accumulate the footfall
footfall = list(df.loc[:, ['datetime', 'footfall']].groupby(['datetime']).sum()['footfall'])
city_df['footfall'] = footfall

In [7]:
# find min and max values from the dataset
footfall_max = city_df['footfall'].max()
footfall_min = city_df['footfall'].min()

In [8]:
# drop any unwanted features
city_df = city_df.drop(columns = ['datetime', 'Temp', 'Humidity' ,'Pressure', 'Rain', 'WindSpeed', 'Rainfall amount (millimetres)',
                                   'Sin_time','Cos_time','Sin_month_num','Cos_month_num','Sin_weekday_num','Cos_weekday_num'])

In [9]:
# split into test, val and train sets
n=len(city_df)

train_df = city_df[0:int(n*0.7)]
val_df = city_df[int(n*0.7):int(n*0.9)]
test_df = city_df[int(n*0.9):]

train_df = train_df.dropna(axis='columns')
val_df = val_df.dropna(axis='columns')
test_df = test_df.dropna(axis='columns')

In [10]:
train_df

,footfall
0,5880
1,8111
2,10310
3,8079
4,3774
...,...
50324,7302
50325,3110
50326,1490
50327,920


## Time series forecasting

In [11]:
# create a window generator that generates windows of data for input into model training
# also has a plotting routine to check the results

class WindowGenerator():
    def __init__(self, input_width, label_width, shift, train_df=train_df, val_df=val_df, test_df=test_df, label_columns=None):
        
        # Store the raw data.
        self.train_df = train_df
        self.val_df = val_df
        self.test_df = test_df
    
        # Work out the label column indices.
        self.label_columns = label_columns
        if label_columns is not None:
            self.label_columns_indices = {name: i for i, name in
                                        enumerate(label_columns)}
        
        self.column_indices = {name: i for i, name in
                            enumerate(train_df.columns)}
    
        # Work out the window parameters.
        self.input_width = input_width
        self.label_width = label_width
        self.shift = shift
    
        self.total_window_size = input_width + shift
    
        self.input_slice = slice(0, input_width)
        self.input_indices = np.arange(self.total_window_size)[self.input_slice]
    
        self.label_start = self.total_window_size - self.label_width
        self.labels_slice = slice(self.label_start, None)
        self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

    def __repr__(self):
        return '\n'.join([
            f'Total window size: {self.total_window_size}',
            f'Input indices: {self.input_indices}',
            f'Label indices: {self.label_indices}',
            f'Label column name(s): {self.label_columns}'])
    
    def split_window(self, features):
        
        inputs = features[:, self.input_slice, -1:]
        labels = features[:, self.labels_slice, :]
        #print(inputs.shape)
        #inputs[:,:,-1] = (inputs[:,:,-1] - footfall_min) / (footfall_max - footfall_min)

        
        if self.label_columns is not None:
            labels = tf.stack([labels[:, :, self.column_indices[name]] for name in self.label_columns],axis=-1)
    
        # Slicing doesn't preserve static shape information, so set the shapes
        # manually. This way the `tf.data.Datasets` are easier to inspect.
        inputs.set_shape([None, self.input_width, None])
        labels.set_shape([None, self.label_width, None])
    
        return inputs, labels
    
    def make_dataset(self, data):
        data = np.array(data, dtype=np.float32)
        ds = tf.keras.utils.timeseries_dataset_from_array(
            data=data,
            targets=None,
            sequence_length=self.total_window_size,
            sequence_stride=1,
            shuffle=True,
            batch_size=32,)
        ds = ds.map(self.split_window)
        return ds
    
    # a plot function to view a few examples of how the footfall prediction compares to the observed value
    def plot(self, model=None, plot_col='footfall', max_subplots=10):
    
        inputs, labels = self.example
        plt.figure(figsize=(12, 18))
        plot_col_index = self.column_indices[plot_col]
        max_n = min(max_subplots, len(inputs))
        print(max_n)
        for n in range(max_n):
            plt.subplot(max_n, 1, n+1)
            plt.ylabel(f'{plot_col} [normed]')
            plt.plot(self.input_indices, inputs[n, :, plot_col_index-1],
                     label='Inputs', marker='.', zorder=-10)
        
            if self.label_columns:
                label_col_index = self.label_columns_indices.get(plot_col, None)
            else:
                label_col_index = plot_col_index
        
            if label_col_index is None:
                continue
        
            plt.scatter(self.label_indices, labels[n, :, label_col_index],
                        edgecolors='k', label='Labels', c='#2ca02c', s=64)
            if model is not None:
                predictions = model(inputs)
                plt.scatter(self.label_indices, predictions[n,label_col_index],
                          marker='X', edgecolors='k', label='Predictions',
                          c='#ff7f0e', s=64)
        
            if n == 0:
                plt.legend()
        
        plt.xlabel('Time [h]')


In [12]:
# set epochs and function for compile/training the model
MAX_EPOCHS = 200

def compile_and_fit(model, window, patience=3):
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                      patience=patience,
                                                      mode='min')

    model.compile(loss=tf.keras.losses.MeanSquaredError(),
                  optimizer=tf.keras.optimizers.Adam(),
                  metrics=[tf.keras.metrics.MeanAbsoluteError()])

    history = model.fit(window.train, epochs=MAX_EPOCHS,
                        validation_data=window.val,
                        callbacks=[early_stopping],
                       verbose=1)
    return history

In [13]:
# setup window with required parameters
w2 = WindowGenerator(input_width=6, label_width=1, shift=1, label_columns=['footfall'])

In [14]:
example_window = tf.stack([np.array(train_df[:w2.total_window_size]),
                           np.array(train_df[100:100+w2.total_window_size]),
                           np.array(train_df[200:200+w2.total_window_size])])

2024-01-26 13:20:33.046412: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-26 13:20:34.496740: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 30927 MB memory:  -> device: 0, name: Tesla V100-SXM2-32GB-LS, pci bus id: 0000:07:00.0, compute capability: 7.0
2024-01-26 13:20:34.524147: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 30927 MB memory:  -> device: 1, name: Tesla V100-SXM2-32GB-LS, pci bus id: 0000:0b:00.0, compute capability: 7.0


In [15]:
# set to check output looks correct
example_inputs, example_labels = w2.split_window(example_window)

print('All shapes are: (batch, time, features)')
print(f'Window shape: {example_window.shape}')
print(f'Inputs shape: {example_inputs.shape}')
print(f'Labels shape: {example_labels.shape}')

All shapes are: (batch, time, features)
Window shape: (3, 7, 1)
Inputs shape: (3, 6, 1)
Labels shape: (3, 1, 1)


In [16]:
# the WindowGenerator object holds training, validation, and test data. 
# Here add properties for accessing them as tf.data.Datasets using the make_dataset method you defined earlier

@property
def train(self):
    return self.make_dataset(self.train_df)

@property
def val(self):
    return self.make_dataset(self.val_df)

@property
def test(self):
    return self.make_dataset(self.test_df)

@property
def example(self):
    """Get and cache an example batch of `inputs, labels` for plotting."""
    result = getattr(self, '_example', None)
    if result is None:
      # No example batch was found, so get one from the `.train` dataset
      result = next(iter(self.train))
      # And cache it for next time
      self._example = result
    return result

WindowGenerator.train = train
WindowGenerator.val = val
WindowGenerator.test = test
WindowGenerator.example = example

In [17]:
# check each batch before going into training
for example_inputs, example_labels in w2.train.take(1):
    print(f'Inputs shape (batch, time, features): {example_inputs.shape}')
    print(f'Labels shape (batch, time, features): {example_labels.shape}')

Inputs shape (batch, time, features): (32, 6, 1)
Labels shape (batch, time, features): (32, 1, 1)


In [18]:
# setup the LSTM model
lstm_model = tf.keras.models.Sequential([
    # Shape [batch, time, features] => [batch, time, lstm_units]
    tf.keras.layers.LSTM(32, return_sequences=False),
    # Shape => [batch, time, features]
    tf.keras.layers.Dense(units=1)
])

In [19]:
# train the model
history = compile_and_fit(lstm_model, w2)

Epoch 1/200


2024-01-26 13:20:38.556844: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8401


1573/1573 [==============================] - 10s 4ms/step - loss: 220090256.0000 - mean_absolute_error: 11783.9004 - val_loss: 431458688.0000 - val_mean_absolute_error: 17614.8418
Epoch 2/200
1573/1573 [==============================] - 6s 4ms/step - loss: 219389696.0000 - mean_absolute_error: 11754.1465 - val_loss: 430379744.0000 - val_mean_absolute_error: 17584.2031
Epoch 3/200
1573/1573 [==============================] - 7s 5ms/step - loss: 218684976.0000 - mean_absolute_error: 11724.1230 - val_loss: 429302048.0000 - val_mean_absolute_error: 17553.5234
Epoch 4/200
1573/1573 [==============================] - 8s 5ms/step - loss: 217971072.0000 - mean_absolute_error: 11693.6572 - val_loss: 428255648.0000 - val_mean_absolute_error: 17523.6895
Epoch 5/200
1573/1573 [==============================] - 6s 4ms/step - loss: 217278096.0000 - mean_absolute_error: 11663.9912 - val_loss: 427219104.0000 - val_mean_absolute_error: 17494.0957
Epoch 6/200
1573/1573 [==============================] -

In [ ]:
# produce some qualitative results
w2.plot(lstm_model)